In [2]:
from dotenv import load_dotenv
import os
import pandas as pd
import matplotlib.pyplot as plt

from eth_rpc import PrivateKeyWallet
from emp_orderly.utils import from_address
from emp_orderly import (
    Strategy, EmpOrderly,
    crossover, plot_heatmaps,
    EMA, SMA, SLOPE, CHOP,
    EmpyrealOrderlySDK,
)
from emp_orderly.onboarding.faucet import request_testnet_funds as faucet
from emp_orderly.onboarding import deposit
from emp_orderly_types import PerpetualAssetType, Interval, OrderType

load_dotenv()

/home/ademola/.cache/pypoetry/virtualenvs/balo-orderly-submission-oa8W0BMU-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [3]:
wallet = PrivateKeyWallet.create_new()
orderly_id = from_address(wallet.address)

sdk = EmpyrealOrderlySDK(pvt_hex=wallet.private_key, account_id=orderly_id, is_testnet=True)

In [4]:
print(wallet.address)

0x42BbA81d923e48bF53C19d113c21e6b565084709


In [5]:
print(await wallet.balance())

0


In [6]:
from alpaca_trade_api import REST
from datetime import timedelta, datetime

ALPACA_API_KEY = os.getenv('ALPACA_API_KEY')
ALPACA_API_SECRET = os.getenv('ALPACA_API_SECRET')
BASE_URL = "https://api.alpaca.markets"
print(ALPACA_API_KEY, ALPACA_API_SECRET)

ALPACA_CREDS = {
    "API_KEY":ALPACA_API_KEY, 
    "API_SECRET": ALPACA_API_SECRET, 
    "PAPER": True
}

class EMSB(Strategy):
    """EMA Momentum and Slope Breakout Strategy.
    
    order_size: The amount of order that can be 
        made at a time.
    days_length_1: 12 days length for 
        calculating 12 days EMA.
    days_length_2: 26 days length for 
        calculating 26 days EMA.
    length: The number of days the Strategy is 
        intended to run for.
    """

    order_size: float = 0.5
    days_length_1: int = 12
    days_length_2: int = 26
    length: int = 14
    symbol: str = "BTC/USD"
    
    @classmethod
    def update_lags(cls, days_length_1, days_length_2, length):
        cls.days_length_1 = days_length_1
        cls.days_length_2 = days_length_2
        cls.length = length

    def get_dates(self): 
        today = datetime.today()
        three_days_prior = today - timedelta(days=14)
        return today.strftime('%Y-%m-%d'), three_days_prior.strftime('%Y-%m-%d')

    def get_news(self):
        today, three_days_prior = self.get_dates()
        news = self.api.get_news(symbol=self.symbol,
                                 start=three_days_prior, 
                                 end=today)
        news = [ev.__dict__["_raw"]["headline"] for ev in news]
        # probability, sentiment = estimate_sentiment(news)
        # return probability, sentiment
        return news

    def init(self):
        close = self.data.close
        self.sma_days_1 = self.I(EMA, close, self.days_length_1)
        self.sma_days_2 = self.I(EMA, close, self.days_length_2)
        self.api = REST(base_url=BASE_URL, key_id=ALPACA_API_KEY, secret_key=ALPACA_API_SECRET)

    def next(self):
        news = self.get_news()
        print(news)
        if (
            crossover(self.sma_days_1, self.sma_days_2)
        ):
            self.position.close()
            self.buy(size=self.order_size)
        elif (
            crossover(self.sma_days_2, self.sma_days_1)
        ):
            self.position.close()
            self.sell(size=self.order_size)


tester = EmpOrderly(
    cash=1000,
    commission=.0001,
    exclusive_orders=True,
    sdk=sdk,
)

# load strategy and data
tester.set_strategy(EMSB)
await tester.load_data(
    lookback=14,
    interval=Interval.five_minute,
    asset=PerpetualAssetType.BTC,
)
# print(tester._history[tester._history["volume"] > 0])

# backtest
tester.backtest()

AKGQXLI45FFBANSEDKWN vGc5bE8PQ3nphKpRSg9XaIaw5C4KbHjKEsoFB3PC


/home/ademola/.cache/pypoetry/virtualenvs/balo-orderly-submission-oa8W0BMU-py3.12/lib/python3.12/site-packages/emp_orderly/backtest/backtesting.py:1175: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  self._init_data(self._history)


["Pay Taxes With Bitcoin? Ohio Senator Proposes Bill For Crypto Tax Payments, But There's A Catch", "Bitcoin, Ethereum, Dogecoin Close September With A Down Day: 'Nothing Goes Up Forever, Now Market Looks Much Healthier,' Says Trader", "Edward Snowden Cautions Crypto Industry Not To Dilute Principles: 'We Should Defy Bureaucracy'", 'Bitcoin Down More Than 3% Within 24 hours', "Bitcoin Miner CleanSpark Bets Big On Bull Market: 'We Could See Bitcoin Peak Just Under $200K,' Says CEO", "Bitcoin Price Is An 'Opportunity,' Technical Analyst Tells Benzinga: Watch These Indicators For Signs Of A Price Surge", "Whale Trader Recaps $7M Profit In September, Predicts Bitcoin To Hit $100K-$200K: 'Leg 2 Is Beginning, Time To Lock In'", "Bitcoin Back Below $64K: Watch This 'Line In The Sand' For Directional Bias, Says Veteran Trader", 'FOMO Drives Biggest Gain In Shanghai Since 2008, Chinese Market Breaks The Negative Pattern', "Bitcoin Up 8% In September: What Traders And Analysts Expect For 'Best-P

In [14]:
# # plot
# tester.plot(show_price_data=False)
# plt.show()